In [ ]:
pip install -r requirements.txt

## EJECUTAR

In [1]:
from langchain_openai import AzureChatOpenAI
from langchain.tools import tool
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent
from langgraph.graph import StateGraph, START, END, Graph 
from models.database import *
from models.models import *
import langgraph
import os
from sqlalchemy.orm import Session
import json
from tools import *
#from dotenv import load_dotenv

In [2]:
#load_dotenv()
MODELO = os.getenv("MODELO")
KEY = os.getenv("KEY")
VERSION = os.getenv("VERSION")
ENDPOINT = os.getenv("ENDPOINT")

MAIL_ENVIO = os.getenv("MAIL_ENVIO")

In [3]:
llm = AzureChatOpenAI(
    api_key = KEY,
    api_version = VERSION,
    azure_endpoint = ENDPOINT,
    azure_deployment = MODELO
    )

## AQUI YA NO

In [ ]:
import openai
openai_client = openai.AzureOpenAI(
    api_key = KEY,
    api_version = VERSION,
    azure_endpoint = ENDPOINT
    )

In [ ]:
messages=[{"role": "system", "content": "Eres un asistente útil."},
        {"role": "user", "content": "¿Cuál es la capital de España?"}]

response = openai_client.chat.completions.create(
    model = MODELO,
    messages = messages,
    max_tokens=100
    )
response.choices[0].message.content

    ("pendiente_condicionada", "flujo"),
    ("pendiente_no_condicionada", "flujo"),
    ("grabar_cesta_srm", "flujo"),
    ("solicitar_firma_inf_prov_cond", "flujo"),
    ("firma_inf_prov_cond", "flujo"),
    ("lanzar_cesta_srm", "flujo"),
    ("aprobar_compra_manager", "aprobacion"),
    ("aprobar_compra_gestion", "aprobacion"),
    ("revision_compra", "flujo"),
    ("solicitar_oferta_provedoores", "flujo"),
    ("proveedor_envia_ofertn", "flujo"),
    ("consulta_acep_oferta", "flujo"),
    ("negociacion_proveedores", "flujo"),
    ("proponer_proveedor", "flujo"),
    ("aprobar_adjuducacion", "aprobacion"),
    ("facturar", "flujo"),
    ("fin", "flujo"),

In [ ]:
# Agrego un proveedor de prueba

def agregar_proveedor_prueba():
    """
    Agrega un proveedor de prueba a la base de datos.
    """
    session = Session(bind=engine)

    # Crear un proveedor de prueba
    nuevo_proveedor = Proveedor(
        nombre="Apple",
        contacto="Soporte Apple",
        email="apple@proveedor.com",
        estado="aprobado"
    )
    session.add(nuevo_proveedor)
    session.commit()

    print(f"✅ Proveedor de prueba añadido: {nuevo_proveedor.nombre} (ID: {nuevo_proveedor.id_proveedor})")
    session.close()

# Ejecutar la función para agregar el proveedor
agregar_proveedor_prueba()


PENDIENTE QUE SE GENERE UNA DESCRIPCION AUTOMATICA POR LA IA Y LE PONGA NOMBRE A LA CESTA EL AGENTE OBTENER DATOS PEDIDO.

## NODO 1 - DATOS PEDIDO 
Este nodo se encarga de obtener los datos del pedido asigna el estado de pedido condicionado o no condicionado en funcion de si encuentra o no datos en la tabla pedidos.

In [ ]:
def obtener_datos_pedido(entrada_usuario):
    ''' Nodo que usa IA para obtener los datos del pedido del usuario, crea la cesta y el pedido con el estado inicial,
      y decide si el flujo valida el proveedor o graba la cesta.
    '''
    # Se crea la sesión de SQLAlchemy	
    session = Session(bind=engine)

    prompt = f''' 
    Eres el encargado de obtener la información de un pedido.
    Se necesita conocer:
    - Nombre del proveedor (si no se menciona, deja "desconocido")
    - Moneda en la que se realizó la compra utiliza el codigo de 3 letras
    - Presupuesto estimado
    - Producto a comprar

    {entrada_usuario}

    Devuelve los datos en un formato claro como:
    "Proveedor: Proveedor X, Moneda: EUR, Presupuesto: 5000, Producto: [Producto Y]"
    '''
    response = llm.invoke(prompt)
    datos_ia = response.content.strip()

    print(f'IA: {datos_ia}')

    # Extraer los datos del texto
    datos_extraidos = {}
    for item in datos_ia.split(','):
        key, value = item.strip().split(':')
        datos_extraidos[key.lower().strip()] = value.strip()
    
    nombre_proveedor = datos_extraidos.get('proveedor', 'desconocido')
    codigo_moneda = datos_extraidos.get('moneda', 'EUR') # Por defecto EUR
    presupuesto = float(datos_extraidos.get('presupuesto', 1000)) # Por defecto 1000

    print(f"📦 Datos extraídos: Proveedor: {nombre_proveedor}, Moneda: {codigo_moneda}, Presupuesto: {presupuesto}")
    
    # Verificar si el proveedor ya existe en la BD
    proveedor = session.query(Proveedor).filter_by(nombre=nombre_proveedor).first()
    proveedor_id = proveedor.id_proveedor if proveedor else None
    # Si el proveedor existe en la BD se va por el flujo de proveedor no condicionado
    if proveedor:
        print(f"✅ Proveedor '{proveedor.nombre}' encontrado en la base de datos.")
        es_nuevo = False
        estado_inicial_nombre = "pendiente_no_condicionada"
    # Si el proveedor no existe en la BD se va por el flujo de proveedor condicionado
    else:
        print(f"⚠️ Proveedor '{nombre_proveedor}' no encontrado. Requiere validación.")
        es_nuevo = True
        estado_inicial_nombre = "pendiente_condicionada"
        entrada_usuario = input("👉 Nombre, email y contacto del proveedor (en lenguaje natural):\n")
        agente_proveedor = create_react_agent(llm ,tools=[insertar_proveedores])
        # Se crea un agente que se encargara de obtener los datos del proveedor e introducirlo en la base de datos
        prompt = f''' 
        Eres el encargado de obtener la información de un proveedor.
        Se necesita conocer:
        - Nombre del proveedor 
        - Mail del proveedor
        - Contacto del proveedor

        Aqui tienes la información que tienes que obtener:

        {entrada_usuario}

        Devuelve unicamente un formato JSON como:
            "nombre": "...",
            "email": "...",
            "contacto": "..."
        '
        Una vez generado el json, utiliza la herramienta insertar_proveedores para insertar el proveedor en la base de datos y no generes ningun mensaje adicional.
        '''
        response = agente_proveedor.invoke({'messages': [HumanMessage(content = prompt)]})
        print(response['messages'][-1].content)

        # 🔁 Volver a buscar el proveedor recién insertado
        proveedor = session.query(Proveedor).filter_by(nombre=nombre_proveedor).first()
        proveedor_id = proveedor.id_proveedor if proveedor else None


    # Obtener estado inicial correspondiente
    estado_inicial = session.query(Estado).filter_by(nombre=estado_inicial_nombre).first()
    if not estado_inicial:
        print(f"⚠️ Estado '{estado_inicial_nombre}' no encontrado en la base de datos.")
        return {"error": "Estado inicial no encontrado"}

    # Verificar que la moneda existe en la BD
    moneda = session.query(Moneda).filter(Moneda.codigo == codigo_moneda).first()
    moneda_id = moneda.id_moneda if moneda else None

    if not moneda:
        print(f"⚠️ Moneda '{codigo_moneda}' no encontrada. Se usará EUR por defecto.")
        moneda = session.query(Moneda).filter_by(codigo="EUR").first()
        moneda_id = moneda.id_moneda

    # Crear la cesta y el pedido
    nueva_cesta = Cesta(
        nombre="Cesta generada automáticamente",
        descripcion="Pedido generado automáticamente",
        tipo_compra="ordinaria",
        presupuesto=presupuesto,
        usuario_sap_id=1,
        proveedor_id=proveedor_id,
        moneda_id=moneda_id,
        contrato_id=None
    )
    session.add(nueva_cesta)
    session.commit()

    print(f"✅ Cesta creada con ID: {nueva_cesta.id_cesta}")

    nuevo_pedido = Pedido(
        posicion=1,
        tipo="Ordinario",
        pedido_tipoimp="K",
        descripcion=f"Compra de {datos_extraidos.get('productos', 'productos no especificados')}",
        id_proveedor=proveedor_id,
        cesta_id=nueva_cesta.id_cesta,  # La Cesta ya está asignada
        estado_tramitacion_id=estado_inicial.id_estado,
        creador_id=1,
        moneda_id=moneda_id
    )
    session.add(nuevo_pedido)
    session.commit()

    # Se extraen las variables antes de cerrar la sesion 

    pedido_id = nuevo_pedido.id_pedido
    cesta_id = nueva_cesta.id_cesta
    session.close()

    return {

        "id_pedido": pedido_id,
        "id_proveedor": proveedor_id, # Ya no intenta acceder a un objeto desvinculado
        "nombre_proveedor": nombre_proveedor,
        "id_cesta": cesta_id,
        "id_moneda": moneda_id,  # Ya no intenta acceder a un objeto desvinculado
        "estado_actual": estado_inicial_nombre,
        "id_usuario": 1,
        "nombre_cesta": "Cesta generada automáticamente",
        "tipo_compra": "ordinaria",
        "presupuesto": presupuesto,
        "descripcion_pedido": "Pedido generado automáticamente",
        "pedido_tipoimp": "K",
        "es_nuevo": es_nuevo  # Indica si el proveedor debe validarse antes de continuar
    }


## NODO 2 - DECIDE EL FLUJO EN FUNCION DE SI EL PEDIDO ESTA CONDICIONADO O NO


In [5]:
def decidir_siguiente_nodo(datos):
    """
    Determina si el flujo sigue a `grabar_cesta_srm` o `gestionar_proveedor_nuevo`
    dependiendo de si el proveedor es nuevo o no.
    """

    # 📌 Debugging: Imprimir el tipo de datos para asegurarnos de que es un diccionario
    print(f"📌 Tipo de datos recibido en decidir_siguiente_nodo: {type(datos)}")

    # 📌 Si `datos` es una tupla, tomar el segundo elemento
    if isinstance(datos, tuple):
        datos = datos[1]  # 🔹 Solo nos interesa la segunda parte de la tupla

    # 📌 Validar si el proveedor es nuevo
    if datos.get("es_nuevo", False):
        print("🔄 Redirigiendo a `gestionar_proveedor_nuevo`...")
        return "gestionar_proveedor_nuevo", datos
    else:
        print("✅ Redirigiendo a `grabar_cesta_srm`...")
        return "grabar_cesta_srm", datos


## NODO 3 - FLUJO DE PEDIDO CONDICIONADO

En este caso se tienen ya los datos del proveedor, por lo que se procede a grabar la cesta en el SRM. Como no se tiene acceso a SRM, se genera un envio de datos en formato JSON, simulando esta comunicación.

PENDIENTE REVISAR LO QUE SE NECESITA EN EL SIGUIENTE NODO.

In [6]:
def grabar_cesta_srm(datos):
    """
     Simula el envío del pedido a SAP SRM, registra el estado en la BD y actualiza `HistorialPedido`
    antes de cambiar el estado a `aprobar_compra_manager`.
    """
    session = Session(bind=engine)

    if isinstance(datos, tuple):
        datos = datos[1]


    id_pedido = datos["id_pedido"]

    #  Obtener el Pedido de la base de datos
    pedido = session.query(Pedido).filter_by(id_pedido=id_pedido).first()
    
    if not pedido:
        print(f"❌ Error: No se encontró el Pedido {id_pedido}.")
        return {"error": "Pedido no encontrado"}

    #  Guardar el estado inicial del Pedido antes de enviarlo a SAP
    estado_anterior_id = pedido.estado_tramitacion_id

    #  Crear JSON para enviar a SAP SRM
    data_sap = {
        "pedido_id": pedido.id_pedido,
        "descripcion": pedido.descripcion,
        "tipo": pedido.tipo,
        "proveedor_id": pedido.id_proveedor,
        "moneda_id": pedido.moneda_id,
        "cesta_id": pedido.cesta_id
    }

    json_sap = json.dumps(data_sap, indent=4)  # 🔹 Simulación de envío
    print(f"📤 Simulando envío de Pedido {id_pedido} a SAP SRM en formato JSON:\n{json_sap}")

    response_data = {
        "status": "success",
        "message": "Pedido registrado correctamente en SAP SRM"
    }

    print(f"✅ Pedido {id_pedido} registrado en SAP SRM (simulación).")
    

    # Se cambia el estado de la tabla pedido a aprobar compra manager
    nuevo_estado = session.query(Estado).filter_by(nombre="grabar_cesta_srm").first()
    if not nuevo_estado:
        print("❌ Error: No se encontró el estado 'grabar_cesta_srm'.")
        return {"error": "Estado no encontrado"}
    
    # Se obtiene la transicion correspondiente en la tabla transiciones
    transicion = session.query(Transicion).filter_by(
        estado_origen_id=estado_anterior_id,
        estado_destino_id=nuevo_estado.id_estado
    ).first()

    if not transicion:
        print(f"⚠️ Advertencia: No se encontró una transición desde {estado_anterior_id} a {nuevo_estado.id_estado}.")
        return {"error": "Transición no encontrada"}

    # Se registra el cambio en el historial del pedido
    historial = HistorialPedido(
        id_pedido=pedido.id_pedido,
        estado_anterior=estado_anterior_id,  # 🔹 Estado antes de SAP
        estado_nuevo=nuevo_estado.id_estado,  # 🔹 Estado después de SAP
        estado_aprobacion="pendiente",
        id_usuario=pedido.creador_id,
        id_transicion=transicion.id_transicion
    )
    session.add(historial)
    session.commit()

    print(f" Cambio registrado en HistorialPedido para Pedido {id_pedido}.")
    
    #  Ahora cambiar el estado del Pedido
    pedido.estado_tramitacion_id = nuevo_estado.id_estado
    session.commit()

    print(f"🔄 Pedido {id_pedido} actualizado al estado '{nuevo_estado.nombre}'.")

    session.close()

    return {
        "id_pedido": id_pedido,
        "estado_actual": nuevo_estado.nombre,
    }

## NODO 4 - FLUJO DE PROVEEDOR CONDICIONADO
Este nodo es importante ya que es necesario obtener los datos del proveedor y enviar generar un pdf tipo con el informe del nuevo proveedor hacia dos departamentos para la aprobación de un nuevo proveedor en el sistema.

QUEDA PENDIENTE AÑADIR EL ID  DE LA TRANSICIÓN

In [ ]:
def gestionar_proveedor_nuevo(datos):
    from datetime import datetime
    session = Session(bind=engine)
    
    nombre_proveedor = datos["nombre_proveedor"]
    email = MAIL_ENVIO
    id_pedido = datos["id_pedido"]
    filename = f"informe_proveedor_{nombre_proveedor}.pdf"

    # 👉 Tools que el agente puede usar
    tools = [generar_pdf_proveedor, enviar_correo]

    # Crear el agente con las tools
    agent_pedido = create_react_agent(llm, tools)

    # 🧠 Prompt que le dice al agente qué hacer
    prompt = f"""
    El proveedor {nombre_proveedor} es nuevo. Debes generar un informe PDF con la información del proveedor
    y enviarlo a {email} para su validación. 

    Usa la herramienta `generar_pdf_proveedor` para crear el informe en el archivo llamado "{filename}" 
    con los datos disponibles del proveedor: {datos}

    Después, usa la herramienta `enviar_correo` para enviar el informe. En el cuerpo del correo justifica 
    por qué este proveedor debe ser considerado (por ejemplo, disponibilidad, precio o características técnicas).

    No devuelvas ningún texto adicional.
    """

    print("🤖 Ejecutando agente para gestionar proveedor nuevo...")
    agent_pedido.invoke({'messages': [HumanMessage(content = prompt)]})

    # 🔄 Actualizar estado y registrar historial
    pedido = session.query(Pedido).filter_by(id_pedido=id_pedido).first()

    if not pedido:
        print(f"❌ Pedido {id_pedido} no encontrado.")
        return {"error": "pedido no encontrado"}

    estado_anterior_id = pedido.estado_tramitacion_id
    nuevo_estado = session.query(Estado).filter_by(nombre="solicitar_firma_inf_prov_cond").first()

    if not nuevo_estado:
        print("❌ Estado 'solicitar_firma_inf_prov_cond' no encontrado.")
        return {"error": "estado no encontrado"}

    # Actualizar estado
    pedido.estado_tramitacion_id = nuevo_estado.id_estado
    session.commit()

    # Registrar en historial
    historial = HistorialPedido(
        id_pedido=id_pedido,
        estado_anterior=estado_anterior_id,
        estado_nuevo=nuevo_estado.id_estado,
        estado_aprobacion="pendiente",
        fecha_cambio=datetime.utcnow(),
        id_usuario=pedido.creador_id,
        id_transicion=1  # Si sabes el ID exacto de esta transición
    )
    session.add(historial)
    session.commit()
    session.close()

    print(f"✅ Pedido {id_pedido} actualizado a solicitar_firma_inf_prov_cond y registrado en Historial.")
    return datos


In [5]:
datos = {
    "id_pedido": 1,
    "nombre_proveedor": "Tech Solutions SL",
    "id_proveedor": 1,
    "id_cesta": 1,
    "id_moneda": 1,  # EUR
    "estado_actual": "pendiente_condicionada",
    "id_usuario": 1,
    "nombre_cesta": "Cesta generada automáticamente",
    "tipo_compra": "ordinaria",
    "presupuesto": 8500.0,
    "descripcion_pedido": "Compra de 20 estaciones de trabajo para el equipo de desarrollo",
    "pedido_tipoimp": "K",
    "es_nuevo": True
}


In [ ]:
def aprobacion_proveedor():
    tools = []
    agent_pedido = create_react_agent(llm, tools)
    print("PENDIENTE APROBACION PROVEEDOR")

def aprobar_compra_manager():
    tools = []
    agent_pedido = create_react_agent(llm, tools)
    print("PENDIENTE APROBACION PROVEEDOR")

In [18]:
# Codigo para generar el flujo

#graph_builder = StateGraph(dict)
graph_builder = Graph()

# Nodos al grafo

graph_builder.add_node("obtener_datos_pedido", obtener_datos_pedido)
graph_builder.add_node("decidir_siguiente_nodo", decidir_siguiente_nodo)
graph_builder.add_node("gestionar_proveedor_nuevo", gestionar_proveedor_nuevo)
graph_builder.add_node("grabar_cesta_srm", grabar_cesta_srm)
graph_builder.add_node("aprobar_compra_manager", aprobar_compra_manager)
graph_builder.add_node("aprobacion_proveedor", aprobacion_proveedor)


In [19]:
# Empieza en obtener datos del pedido
graph_builder.add_edge(START, "obtener_datos_pedido")

graph_builder.add_edge("obtener_datos_pedido", "decidir_siguiente_nodo")

# 📌 Configurar la decisión basada en la existencia del proveedor
graph_builder.add_conditional_edges("decidir_siguiente_nodo", decidir_siguiente_nodo)

# 📌 Conectar `gestionar_proveedor_nuevo` con `grabar_cesta_srm` para continuar el flujo
graph_builder.add_edge("gestionar_proveedor_nuevo", "grabar_cesta_srm")

# Termina en grabar cesta srm
graph_builder.add_edge("grabar_cesta_srm", END)

In [20]:
from langchain_core.tools import tool

In [21]:
graph = graph_builder.compile()

In [ ]:
prompt = "Quiero comprar 20 portatiles del proveedor samsung con un presupuesto de 6600 euros"
result = graph.invoke(prompt)

IA: Proveedor: Samsung, Moneda: EUR, Presupuesto: 6600, Producto: Portátiles
📦 Datos extraídos: Proveedor: Samsung, Moneda: EUR, Presupuesto: 6600.0
2025-03-23 18:26:20,487 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-23 18:26:20,491 INFO sqlalchemy.engine.Engine SELECT proveedores.id_proveedor AS proveedores_id_proveedor, proveedores.nombre AS proveedores_nombre, proveedores.contacto AS proveedores_contacto, proveedores.email AS proveedores_email, proveedores.estado AS proveedores_estado, proveedores.fecha_registro AS proveedores_fecha_registro 
FROM proveedores 
WHERE proveedores.nombre = ?
 LIMIT ? OFFSET ?
2025-03-23 18:26:20,492 INFO sqlalchemy.engine.Engine [cached since 1.88e+04s ago] ('Samsung', 1, 0)
⚠️ Proveedor 'Samsung' no encontrado. Requiere validación.


In [ ]:
MAIL_KEY = os.environ.get("MAIL_KEY")
MAIL = os.environ.get("MAIL")

In [ ]:
import imaplib
import email
from email.header import decode_header

def leer_bandeja_entrada(usuario, contraseña, servidor="imap.gmail.com", n=5):
    """
    Lee los últimos n correos de la bandeja de entrada.
    
    Parámetros:
    - usuario: tu dirección de correo
    - contraseña: contraseña o token de aplicación (para Gmail)
    - servidor: servidor IMAP (por defecto Gmail)
    - n: número de correos a leer
    """

    # Conectarse al servidor
    mail = imaplib.IMAP4_SSL(servidor)
    mail.login(usuario, contraseña)
    mail.select("inbox")

    # Buscar todos los correos
    _, mensajes = mail.search(None, "ALL")
    mensajes = mensajes[0].split()

    print(f"📥 Total de correos: {len(mensajes)}\n")

    for i in mensajes[-n:]:
        _, data = mail.fetch(i, "(RFC822)")
        mensaje = email.message_from_bytes(data[0][1])

        # Obtener asunto
        asunto, cod = decode_header(mensaje["Subject"])[0]
        if isinstance(asunto, bytes):
            asunto = asunto.decode(cod or "utf-8")

        # Obtener remitente
        de = mensaje.get("From")

        print(f"📧 Asunto: {asunto}")
        print(f"👤 De: {de}")

        # Mostrar contenido
        cuerpo = ""
        if mensaje.is_multipart():
            for parte in mensaje.walk():
                tipo = parte.get_content_type()
                disp = str(parte.get("Content-Disposition"))
                if tipo == "text/plain" and "attachment" not in disp:
                    cuerpo = parte.get_payload(decode=True).decode()
                    break
        else:
            cuerpo = mensaje.get_payload(decode=True).decode()

        print("📨 Contenido:")
        print(cuerpo[:500])  # Mostrar los primeros 500 caracteres
        print("=" * 50)

    mail.logout()


In [ ]:
leer_bandeja_entrada(MAIL, MAIL_KEY, n=3)


📥 Total de correos: 3

📧 Asunto: We’re
👤 De: Legal Customer Notices <legal-customer-notices@twilio.com>
📨 Contenido:
https://www.twilio.com/login

Ahoy!

You are receiving this email from Twilio because you have an active Twilio or Twilio SendGrid account, or you are a Twilio partner.

What do you need to know?

We updated our terms linked below, which will automatically be effective as of April 1, 2025. We encourage you to take a look at our detailed notice https://www.twilio.com/legal/online-terms-updates, which highlights the main changes, as well as the updated terms themselves, so you can stay inf
📧 Asunto: Consulta la configuración de privacidad de tu cuenta de Google
👤 De: Google <no-reply@accounts.google.com>
📨 Contenido:
[image: Google]



Gestiona las opciones de privacidad de tu cuenta con la Revisión de
Privacidad
blueprismautomail@gmail.com


Es un buen momento para revisar la configuración de privacidad de tu cuenta
de Google. Consulta y gestiona los datos que usa Google 

In [ ]:
def generar_pdf_proveedor(filename, datos):
    """
    Genera un archivo PDF con los datos de un proveedor.
    
    Parámetros:
    - filename: Nombre del archivo PDF.
    - datos: Diccionario con los datos del proveedor.
    """
    pdf = FPDF()
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.add_page()
    pdf.set_font("Arial", style='', size=12)

    pdf.cell(200, 10, "Informe de Nuevo Proveedor", ln=True, align="C")
    pdf.ln(10)

    pdf.cell(200, 10, f"Proveedor: {datos['nombre_proveedor']}", ln=True)
    pdf.cell(200, 10, f"ID Pedido: {datos['id_pedido']}", ln=True)
    pdf.cell(200, 10, f"Presupuesto: {datos['presupuesto']} {datos['id_moneda']}", ln=True)

    pdf.output(filename)
    print(f"📄 PDF generado: {filename}")

In [3]:
from fpdf import FPDF

In [5]:
generar_pdf_proveedor("informe.pdf", {"nombre_proveedor": "Apple", "id_pedido": 1, "presupuesto": 5000, "id_moneda": "EUR", "tipo_compra": "ordinaria"})

📄 PDF generado: informe.pdf
